## Import Libraries

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from matplotlib.ticker import StrMethodFormatter, NullFormatter
%matplotlib inline
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
from metpy.units import units

In [ ]:
### define constants for calculations
cp=1004.; Rd=287.; Rv=461.5; eps = Rd/Rv; L = 2.5e6

def open_iMet(imet_filepath):
    '''Opens iMet Sounding to a pandas dataframe'''
    df = pd.read_table(open(imet_filepath, errors='ignore'),   ###reading in file as a pandas dataframe
                  skiprows=[1,2], sep=r'\s+', index_col = False)
    df['Temp'] = df['Temp']+273.15 ###Converts to K
    
    return df
        

## Get radiosonde file for Figure S1

In [ ]:
FilePath = '/Users/cneumaie/Desktop/Research/BACS_Modeling/Soundings/*.txt'
files = sorted(glob(FilePath))
df = open_iMet(files[0])

## Plot Figure S1

In [ ]:
SavePath = '/Users/cneumaie/Desktop/Research/BACS_Modeling/Soundings/'
plt.rcParams['figure.figsize'] = (14, 14)
plt.rcParams['font.size'] = 40
plt.rcParams['font.family'] = 'Verdana'
plt.rcParams['text.color'] = 'black'
plt.rcParams['lines.linewidth'] = 2
i = 50
plt.figure(dpi = 300)
res2 = 25
skew = SkewT(rotation = 45)
date = str(df['UTC_Date'][0]) ###get date of launch
date = date.replace('/', '')
time = str(df['UTC_Time'][0]) ##get time of launch in UTC
p = df['Press'].rolling(i, center = True).mean().values*units.mbar ### Pressure, using metpy to get units on variable to do metpy sounding
p1 = df['Press'].rolling(i, center = True).mean().dropna()
z = df['Alt_AGL'].rolling(i, center = True).mean()   ###Height Above Ground Level (m)
t = df['Temp'].rolling(i, center = True).mean().apply(lambda x: np.sum(x * p1)/np.sum(p1)) -273.15 ### Temperature (C)
td = df['DP'].rolling(i, center = True).mean().apply(lambda x: np.sum(x * p1)/np.sum(p1)) ### Dewpoint Temperature (C)
rh = df['RelHum'] ###Relative Humidity (%)
wind_speed = df['WSpeed']  ### Wind Speed (m/s)
wind_dir = df['WDirn'].rolling(i, center = True).mean().apply(lambda x: np.sum(x * p1)/np.sum(p1)).values  * units.degrees    ## Wind Direction (Degrees from North)
wind_speed1 = df['WSpeed'].rolling(i, center = True).mean().apply(lambda x: np.sum(x * p1)/np.sum(p1)).values *1.94384* units.knots## This wind speed is converted to knots for sounding
wind_speed_calc = units.Quantity(wind_speed1, 'knots') ### This converts wind speed so we can get u,v in the next line
u, v = mpcalc.wind_components(wind_speed1, wind_dir)   ### This converts wind speed and direction into u,v vectors for sounding
ar = df['Ascent']   ### Ascent Rate (m/s)
mr = df['Mix_Rat'] ### Mixing Ratio (g/kg)
# print(u,v)
# skew.plot(p1, df['Temp']-273.15, 'k--', alpha = 0.5)
skew.plot(p[::res2],t[::res2], 'r', linewidth = 6)
# skew.plot(p1, df['DP'], 'k--', alpha = 0.5)
skew.plot(p[::res2],td[::res2], 'g', linewidth = 6)

    # skew.plot_barbs(P, u,v)
    # Set spacing interval--Every 50 mb from 1000 to 100 mb
my_interval = np.arange(100, 900, 25)* units('mbar')
# print(t)
    # Get indexes of values closest to defined interval
ix = mpcalc.resample_nn_1d(p, my_interval)
    # Plot only values nearest to defined interval values
res = 120
skew.plot_barbs(p[::res], u[::res], v[::res], length = 10)
# skew.plot_dry_adiabats()
# skew.plot_moist_adiabats()
# skew.plot_mixing_lines()
skew.ax.set_ylim(850, 100)
skew.ax.set_xlim(-50,30)
# plt.title(date + ' ' + time + ' UTC')
plt.ylabel('hPa')
plt.xlabel('$^o$C')
SavePath = '/Users/cneumaie/Downloads/'
plt.savefig(SavePath+'IOP3env_smoothed_CPTrains', dpi = 600)
plt.show()
plt.close()

